In [215]:
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt

mesh = ngs.Mesh('proximity_geo.vol')
mesh.Curve(2)
# mesh.RefineHP(3,factor = 0.2)
mesh.Refine()

In [216]:
# Parameters

circ_mass = np.pi*0.031**2
circ_number = 58

mu0 = 1.256*1e-6
mu_iron = 1000*mu0
sigma = 58*1e6
J = 1e7
eps = 1e-10
order = 2

circles_left = ""; 
for i in range(58): circles_left+="circle_left_"+str(i)+"|"
circles_right = ""; 
for i in range(58): circles_right+="circle_right_"+str(i)+"|"

cf_Js = mesh.MaterialCF({circles_left: J, circles_right: -J}, default = 0)
cf_mu = mesh.MaterialCF({ circles_left + circles_right + "left|right|bridge" : mu0, "ambient": mu_iron})
cf_sigma = mesh.MaterialCF({circles_left + circles_right: sigma}, default = 0)

#############################################################################################################

f = 50 #(Hz)
w = f*2*np.pi

H1 = ngs.H1(mesh, order = order, dirichlet = "ambient_face", complex = True)
PH1 = ngs.NumberSpace(mesh, complex = True)
PH2 = ngs.NumberSpace(mesh, complex = True)
# PH1 = ngs.H1(mesh, order = order, dirichlet = "circle_edge", complex = True, definedon = circles_left + circles_right)

X = H1*PH1*PH2

(A,L,M), (As,Ls,Ms) = X.TnT()
# (A,L), (As,Ls) = X.TnT()

K = ngs.BilinearForm(X)
# K += (1/cf_mu*ngs.grad(A)*ngs.grad(As))*ngs.dx\
#     + ((1j*w)*cf_sigma*A*As)*ngs.dx(circles_left + circles_right)\
#     + (cf_sigma*L*As)*ngs.dx(circles_left + circles_right)\
#     + ((1j*w)*cf_sigma*A*Ls)*ngs.dx(circles_left + circles_right)\
#     + (cf_sigma*L*Ls)*ngs.dx(circles_left + circles_right)

K += (1/cf_mu*ngs.grad(A)*ngs.grad(As))*ngs.dx\
    + ((1j*w)*cf_sigma*A*As)*ngs.dx(circles_left + circles_right)\
    + (cf_sigma*L*As)*ngs.dx(circles_left)\
    + (cf_sigma*M*As)*ngs.dx(circles_right)\
    + ((1j*w)*cf_sigma*A*Ls)*ngs.dx(circles_left)\
    + ((1j*w)*cf_sigma*A*Ms)*ngs.dx(circles_right)\
    + (cf_sigma*L*Ls)*ngs.dx(circles_left)\
    + (cf_sigma*M*Ms)*ngs.dx(circles_right)

K.Assemble()

# rows,cols,vals = K.mat.COO()
# import scipy.sparse as sp
# A = sp.csr_matrix((vals,(rows,cols)))
# plt.spy(A, markersize = 1)

volCopper = ngs.Integrate(ngs.CF(1.0)*ngs.dx(circles_left), mesh)

f = ngs.LinearForm(X)
f += (cf_Js*Ls)*ngs.dx(circles_left)\
    +(cf_Js*Ms)*ngs.dx(circles_right)
f.Assemble()


A_and_L = ngs.GridFunction(X)
A_and_L.vec.data += K.mat.Inverse(X.FreeDofs())*f.vec

A = A_and_L.components[0]
L = A_and_L.components[1]
M = A_and_L.components[2]
B = ngs.grad(A)

tot_J = (1j*w)*cf_sigma*(A) + cf_sigma*(L) + cf_sigma*(M)


# # J = J/intJ*I

In [217]:
# Draw(ngs.Norm(J), mesh, settings = {"Objects": {"Wireframe": False}})
# Draw(ngs.Norm(B),mesh, settings = {"Objects": {"Wireframe": False}})

# Draw(ngs.log(ngs.Norm(J)), mesh, settings = {"Objects": {"Wireframe": False}})
# Draw(ngs.log(ngs.Norm(B)), mesh, settings = {"Objects": {"Wireframe": False}})
Draw(ngs.log(ngs.Norm(tot_J)), mesh, settings = {"Objects": {"Wireframe": False}})
# Draw(tot_J, mesh, settings = {"Objects": {"Wireframe": False}}, max = 1e7, min=0)
Draw(tot_J, mesh, settings = {"Objects": {"Wireframe": False}})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

BaseWebGuiScene

In [218]:
ngs.Integrate(ngs.Norm(tot_J),mesh)

458.6364241121236

In [219]:
ngs.Integrate(1/sigma*ngs.Norm(tot_J)*ngs.Norm(tot_J), mesh)

104.81408098855535

In [220]:
L2 = ngs.L2(mesh, complex = True)
a = ngs.GridFunction(L2)
a.Set(tot_J)
print(max(a.vec.FV().real),max(a.vec.FV().imag))
print(min(a.vec.FV().real),min(a.vec.FV().imag))

38483.33407409862 13712427.82155069
-38449.09072469734 -13724282.14681832


In [221]:
38483*58/2

1116007.0